In [1]:
import tensorflow as tf
import numpy as np
from transformer.hparams import Hparams

/Users/rahulchandran/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# data_load utils:

#1. tokenize using google sentence-piece split vocab 
def load_vocab(vocab_fpath):
    vocab = [line.split()[0] for line in open(vocab_fpath,'r').read().splitlines()]
    token2idx = {token:idx for idx,token in  enumerate(vocab)}
    idx2token = {idx:token for idx,token in enumerate(vocab)}
    return token2idx,idx2token

# load length controlled sentences

def load_data(fpath1,fpath2,maxlen1,maxlen2):
    sents1,sents2 = [],[]
    with open(fpath1,'r') as f1, open(fpath2,'r') as f2:
        for sent1,sent2 in zip(f1,f2):
            if len(sent1.split())+1 > maxlen1:
                continue
            if eln(sent2.split())+1 > maxlen2:
                continue
            sents1.append(sent1.strip())
            sents2.append(sent2.strip())
    return sents1,sents2

def calc_num_batches(total_num,batch_size):
    return total_num // batch_size + int(total_num % batch_size != 0)

def sent2num_encode(inp_sent,type_inp,t2idx_dict):
    inp_str = inp_sent.decode("utf-8")
    if type_inp == "x":
        tokens = inp_str.split() + ["</s>"]
    else:
        tokens= ["<s>"] + inp_str.split() + ["</s>"]
    enc_list = [t2idx_dict.get(t,t2idx_dict["<unk>"]) for t in tokens]
    return enc_list


def gen_fn(sents1,sents2,vocab_fpath):
    token2idx, _ = load_vocab(vocab_fpath) 
    #Now the generator part using yield
    for sent1,sent2 in zip(sents1,sents2):
        x = sent2num_encode(sents1,"x",token2idx)
        y = sent2num_encode(sents2,"y",token2idx)
        
        decoder_inp,y = y[:-1], y[1:]
        x_seq_len, y_seq_len =  len(x),len(y)
        yield (x,x_seq_len,sent1) , (decoder_inp,y,y_seq_len,sent2)


def data_feed_fn(sents1,sents2,vocab_fpath,batch_size,shuffle=False):
    # xs = (x[int32] , x_seq_len[int32], sents1[string])
    # ys = (decoder_inp[int32], y[int32], y_seq_len[int32], sents2[string])
    shapes=(([None],(), ()),([None],[None],(),()))  # represents xs, ys
    
    types = (
    (tf.int32,tf.int32,tf.string),
    (tf.int32,tf.int32,tf.int32,tf.string)
    )
    
    paddings = (
    (0,0,''),
    (0,0,0,'')
    )
    
    dataset = tf.data.Dataset.from_generator(
        gen_fn,
        output_shapes=shapes,
        output_types = types,
        args = (sents1,sents2,vocab_fpath))
    if shuffle:
        dataset = dataset.shuffle(128 * batch_size)
    
    dataset = dataset.repeat() # infinite fetch
    dataset  = dataset.padded_batch(batch_size, shapes, paddings).prefetch(1)
    return dataset

def get_batch(fpath1,fpath2,maxlen1,maxlen2,vocab_fpath,batch_size,shuffle=False):
    sents1,sents2 = load_data(fpath1,fpath2,maxlen1,maxlen2)
    batches = data_feed_fn(sents1,sents2,vocab_fpath,batch_size,shuffle)
    num_batches = calc_num_batches(len(sents1), batch_size)
    return batches, num_batches, len(sents1)
    





In [ ]:
# Useful module functions..
#1. Token_embedding layer
def get_token_embeddings(vocab_sz,d_embed,zero_pad=True):
    with tf.variable_scope("shared_weight_matrix"):
        embeddings = tf.get_variable("weight_mat",
                                     dtype = tf.float32,
                                     shape= (vocab_sz,d_embed),
                                     initializer=tf.contrib.layers.xavier_initializer())
        if zero_pad:
            embeddings = tf.concat((tf.zeros(shape=[1,d_embed]) ,embeddings[1:,:]),0)
    return embeddings


#2. positional encoding as given in the paper 

def positional_encoding(inputs,maxlen,masking=True,scope="positional_encoding"):
    # Get embedding_dims into E : this is a fixed value
    E = inputs.get_shape().as_list()[-1]
    
    # Now get the batch_len [here 1] N  and  seq size T <= maxlen .. Both of these are dynamic , so the tf.shape here
    N, T = tf.shape(inputs)[0],tf.shape(inputs)[1]
    
    with tf.variable_scope(scope,reuse=tf.AUTO_REUSE):
        #Now get positional indices. This is implemented by creating a range(T) ie. [0..T]  , which can then be 
        # used to look for index in embedding. Tile it to create a stack of rows, which will be easy for look up
        
        positional_indices = tf.tile(tf.expand_dims(tf.range(T),0),[N,1]) # Tile N,1 times or [1,1] / 1 times here..
        
        # And that's super smart code.. B-)  
        position_enc = np.array([[pos/ np.power(10000,(i-i%2)/E) for i in range(E)]
                            for pos in range(maxlen)])
        
        #Apply sin to odd positions and cos to even positions in embeddings.. 
        position[:,::2] = np.sin(position_enc[:,::2]) # 2i
        position[:,1::2] = np.cos(position_enc[:,1::2]) # 2i+1
        
        #Convert to tensor to use it in embedding 
        position_enc = tf.convert_to_tensor(position_enc, tf.float32) # (maxlen, E)
        
        
        # lookup
        outputs = tf.nn.embedding_lookup(position_enc, position_ind)
        
        #Masking logic : if inputs contains 0 at that position, make it zero or use outputs
        
        if masking:
            outputs = tf.where(tff.equal(inputs,0) , inputs, outputs)
        
    return tf.to_float(outputs)
    
    
def mask(inputs,queries = None,keys = None,mask_type=None):
    outputs = None
    # define negative infinity
    padding_neg_infinity = - 2**32 + 1
    
    if mask_type == "key": # This is to mask padded value sgetting attention .... w.r.t key 
        # get paddings part, which typically contains 0's in embeddings part
        # MAtch the mask to inputs  which is Q.K having dims (N,T_Q,T_K)
        
        masks =  tf.sign(tf.reduce_sum(tf.abs(keys),axis = -1)) # (N, T_k)
        masks  = tf.expand_dims(masks,axis=1) # (N, 1, T_k)
        masks = tf.tile(masks, [1,tf.shape(queries)[1],1]) # Tile to (N,T_q,T_k)
        
        
        # Now apply masks to inputs with negaive infinity 
        paddings = tf.ones_like(inputs) * padding_neg_infinity
        outputs = tf.where(tf.equal(masks,0),paddings, inputs)
        
    elif mask_type=="query": # This is to mask padded value sgetting attention .... w.r.t query 
        masks = tf.sign(tf,reduce_sum(tf.abs(queries),axis = -1)) #(N, T_q)
        masks = tf.expand_dims(masks,axis=-1) # (N, T_q,1) <- Note the change in logic...So awesome ha.. :P
        masks  = tf.tile(masks,  [1,1,tf.shape(keys)[1]]) # Tile to (N,T_q,T_k)
        outputs = inputs * masks
        
    elif mask_type=="future": # This is to mask the other side [next prediction] getting eyed by the decoder..
        #This is done via lower unit triangle
        outer_dim_set = tf.ones_like(inputs[0,:,:]) # Get (T_q, T_k) values
        low_triangle_diag = tf.linalg.LinearOperatorLowerTriangular(outer_dim_set).to_dense() # (T_q, T_k)
        masks = tf.tile(tf.expand_dims(low_triangle_diag,0),[tf.shape(inputs)[0], 1, 1]) # (N,T_q,T_k)
        paddings = tf.ones_like(inputs) * padding_neg_infinity
        outputs = tf.where(tf.equal(masks, 0), paddings, inputs)
    else:
        print("Check if you entered mask type correctly!")
    return outputs


def scaled_dot_product_attention(Q,K,V,causality=False,
                                 dropout_rate=0.0,training=True,scope="scaled_dot_produt_attention"):
    with tf.variable_scope(scope,reuse=tf.AUTO_REUSE):
        # get embedding dims
        d_k = Q.get_shape().as_list()[-1]
        
        # dot product (Q dot K)
        # Q is (N,T_q,d_k) and K is (N,T_k,d_k)
        outputs = tf.matmul(Q,(tf.transpose(k,[0,2,1])))
        
        # Now scale with sqrt(d_k)
        outputs /=  d_k ** 0.5
        
        # Now do the key masking to remove attention from padded values w.r.t keys
        outputs = mask(outputs, Q,K, mask_type="key")
        
        # Now do it for decoder right side blind masking case
        if causality:
            outputs = mask(outputs, mask_type="future")
            
        # Now find the softmax
        outputs = tf.nn.softmax(outputs)
        attention  = tf.transpose(outputs,[0,2,1])
        tf.summary.image("attention",tf.expand_dims(attention[:1],-1))
        
        
        #query masking
        outputs = mask(outputs,Q,K,mask_type="query")
        
        #Add dropout
        outputs = tf.layers.dropout(outputs, rate=dropout_rate, training=training)
        
        # Now the last remaining output x value part
        outputs = tf.matmul(outputs, V) #(N,T_q,d_v)
        
        
    return outputs
    
def layer_normalize(inputs,epsilon=1e-10, scope="layer_normalize"):
    with tf.variable_scope(scope,tf.AUTO_REUSE):
        vec_dims = inputs.get_shape().as_list[-1:] # : is used to return as array with the last dim
        mean_vec_axis, variance_vec_axis =  tf.nn.moments(inputs,[-1],keep_dims=True)
        beta = tf.get_variable("beta",vec_dims,initializer=tf.zeros_initializer())
        gamma = tf.get_variable("gamma", vec_dims, initializer=tf.ones_initializer())
        normalized = (inputs-mean_vec_axis) / ((variance_vec_axis+epsilon) ** (0.5))
        outputs = gamma * normalized+ beta 
    return outputs


        
def multihead_attention(queries,keys,values,num_heads,dropout_rate=0,
                        training=True,training=True,causality=False,scope="multihead_attention") 
    # Take embedding dim
    d_model = queries.get_shape().as_list()[-1]
    with tf.variable_scope(scope,reuse=tf.AUTO_REUSE):
        
        #Setting W_q, W_k, W_v
        Q = tf.layers.dense(queries,d_model) # N x T x d_model
        K = tf.layers.dense(keys,d_model) # N x T x d_model 
        V = tf.layers.dense(values,d_model) # N x T x d_model
        
        Q_ = tf.split(Q,num_heads,axis=2)
        Q_ = tf.concat(Q_,axis=0) # [(num_heads * N) x T x (d_model/num_heads) ]
        
        K_ = tf.split(K, num_heads,axis=2)
        K_ = tf.concat(K_,axis=0)  # [(num_heads * N) x T x (d_model/num_heads) ]
        
        V_ = tf.split(V, num_heads,aaxis=2)
        V_ = tf.concat(V_, axis=0)
        
        outputs = scaled_dot_product_attention(Q_, K_, V_, causality, dropout_rate, training)

        #Reshape to default
        
        outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2 ) #(N, T_q, d_model)
        
        # Residual connection
        outputs += queries
        
        # Now layer normalization : Effective for recurrent nets than batch normalization 
        # https://arxiv.org/abs/1607.06450
        outputs += layer_normalize(outputs)
        
    return outputs


In [ ]:
# Now comes the Transformer class

class Transformer:
    
    def __init__(self,hp):
        self.hp = hp
        self.token2idx, self.idx2token = load_vocab(hp.vocab)
        self.embeddings = get_token_embeddings(self.hp.vocab_size,self.hp.d_model,zero_pad=True)
        
    
    def encode(self,xs, training=True):
        with tf.variable_scope("encoder",reuse=tf.AUTO_REUSE):
            x, seqlens, sents1 = xs
            
            # Now the embedding part
            enc = tf.nn.embedding_lookup(self.embeddings,x)
            # scale the embedding with sqrt of model_dim
            enc *= self.hp.d_model ** 0.5
            
            # Now add positional encoding using the formulas given in the paper.
            enc += positional_encoding(enc,self.hp.maxlen1)
            
            #Add dropout for embedding layer..
            enc = tf.layers.dropout(enc, self.hp.dropout_rate, training=training)
            
            
            
            
            